<h2>Run posterior sampler on GCMS data</h2>

In [169]:
%load_ext autoreload
%autoreload 2
import sys,os
sys.path.append('../fragments')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [170]:
from network_sampler import *
from frag_set import *

In [171]:
gcms_peaks = FragSet()
gcms_peaks.load_from_file('mzMATCHoutput.txt')
gcms_peaks.load_annotations('temp_nist.txt',correct_gcms_derivatives = True)

Loading from mzMATCHoutput.txt
Loaded 11870 peaks
...with 2272 relations
	Parsing NIST output
	Loaded 263 unique annotations


In [172]:
ns = NetworkSampler(gcms_peaks)

Creating adjacency structure. This might take some time...
Found match: C11H26O3 -> H2O -> C11H24O2
Found match: C11H26O3 -> H2O -> C11H24O2
Found match: C11H26O3 -> H2O -> C11H24O2
Found match: C11H26O3 -> C2H4 -> C9H22O3
Found match: C11H26O3 -> O -> C11H26O2
Found match: C11H26O3 -> O -> C11H26O2
Found match: C14H32O3 -> H2O -> C14H30O2
Found match: C17H29NO3 -> C2H2 -> C19H31NO3
Found match: C18H49NO5 -> O -> C18H49NO6
Found match: C18H49NO5 -> HPO3 -> C18H50NO8P
Found match: C10H27N -> CO -> C11H27NO
Found match: C10H27N -> C5H7NO3 -> C15H34N2O3
Found match: C11H28N2O3 -> C2H2O -> C9H26N2O2
Found match: C10H20O -> H2 -> C10H22O
Found match: C10H20O -> O -> C10H20
Found match: C14H23NO3 -> H2 -> C14H21NO3
Found match: C14H23NO3 -> CO2 -> C13H23NO
Found match: C12H17NO3 -> C2H4 -> C14H21NO3
Found match: C12H17NO3 -> C3H5NO -> C9H12O2
Found match: C12H17NO3 -> NH -> C12H18N2O3
Found match: C18H43NO6 -> C2H4 -> C16H39NO6
Found match: C18H43NO6 -> C5H9NO -> C13H34O5
Found match: C18H43

In [173]:
ns.delta = 1
ns.initialise_sampler()

In [174]:
ns.multiple_network_sample(10000)

In [175]:
ns.dump_output(outstream=open('posteriors.txt','w'))

<p>Find all the things one step from m</p>

In [179]:
for me in ns.peakset.measurements:
    print "\nMeasurement {}".format(me.id)
    ptotal = 0.0
    pcounts = 0
    for an in me.annotations:
        ptotal += me.annotations[an]
        pcounts += ns.peakset.posterior_counts[me][an]
    
    for an in me.annotations:
        print '\t' + str(an.formula) + " {:.20s}".format(an.name) + " Prior: " + str(me.annotations[an]/ptotal) + " Posterior: " + str(1.0*ns.peakset.posterior_counts[me][an]/pcounts)
        for an2 in ns.adjacency[an]:
            trans = ns.get_all_transforms(an,an2)
            print '\t\tAdjacent to: ' + str(an2.formula) + str(trans) + " {:.20s}".format(an2.name)
            adjacent_measurements = [mea for mea in ns.peakset.measurements if an2 in mea.annotations]
            for m2 in adjacent_measurements:
                prior_total = 0.0
                total_counts = 0
                for an3 in m2.annotations:
                    prior_total += m2.annotations[an3]
                    total_counts += ns.peakset.posterior_counts[m2][an3]
                print '\t\t\t Measurement: ' + str(m2.id) + ' Prior: ' + str(m2.annotations[an2]/prior_total) + " Posterior: " + str(1.0*ns.peakset.posterior_counts[m2][an2]/total_counts)


Measurement 0
	C10H18N2O3 2-Amino-4-nitropheno Prior: 1.0 Posterior: 1.0

Measurement 1

Measurement 2
	C14H32O3 �-Hydroxyisobutyric  Prior: 0.17152712449 Posterior: 0.0778
		Adjacent to: C14H30O2[H2O] Silane, methylvinyl(
			 Measurement: 495 Prior: 1.0 Posterior: 1.0
	C11H26O3 2-Trimethylsilyloxyh Prior: 0.82847287551 Posterior: 0.9222
		Adjacent to: C11H24O2[H2O] 4-Methylvaleric acid
			 Measurement: 73 Prior: 0.462089300758 Posterior: 0.5222
		Adjacent to: C11H24O2[H2O] Hexanoic acid, TBDMS
			 Measurement: 73 Prior: 0.279696714406 Posterior: 0.3024
		Adjacent to: C11H24O2[H2O] trans-1,4-Cyclohexan
			 Measurement: 201 Prior: 1.0 Posterior: 1.0
		Adjacent to: C9H22O3[C2H4] 2-Hydroxy-3-methylbu
			 Measurement: 247 Prior: 0.337186897881 Posterior: 0.5306
		Adjacent to: C11H26O2[O] Silane, dimethyl(3-h
			 Measurement: 641 Prior: 1.0 Posterior: 1.0
		Adjacent to: C11H26O2[O] Silane, dimethyl(2-o
			 Measurement: 652 Prior: 0.846370023419 Posterior: 0.8595

Measurement 4
	C17H29NO3 O